In [ ]:
%run common_init.py

Start import


# Direct detection of Dark matter using different target materials #

Author:

Joran Angevaare <j.angevaare@nikef.nl>

Date:

14 october 2019 

## Goal ## 

- Roughly reproduce <https://journals.aps.org/prd/abstract/10.1103/PhysRevD.83.083505>
- Update the results thereof with more recent knowledge of the DM-distribution

### Approach ###
To achieve these goals, we must first get a decent recoil spectrum, that is flexible enough to have different astrophysical parameters. Further, it must also be flexible enough to be able to allow for different analytic functions to be added to the model. For instance, we must be able to incorporate the $\rm{SHM}^{++}$ as presented here <https://arxiv.org/abs/1810.11468>.

When we have a sufficiently flexible model, we want to add in the detector physics, we should therein incorporate at least the following parameters:
- target
  - material
  - cross-section
- threshold
- background
- cut efficiency  
- volume
- exposure

Finally we should perform the inference

In [ ]:
from datetime import datetime

In [ ]:
from pymultinest.solve import solve as multines_solve

In [ ]:
stats = dddm.NestedSamplerStatModel('Xe')

In [ ]:
stats._log_probability_nestle([0,0])

In [ ]:
time.time()

In [ ]:
from pymultinest.solve import solve

In [ ]:
from __future__ import absolute_import, unicode_literals, print_function
import numpy
from numpy import pi, cos

In [ ]:
%time
start_multi = time.time()

result = solve(LogLikelihood=stats._log_probability_nestle, 
      Prior=stats._log_prior_transform_nestle,
      n_dims = 2,
      outputfiles_basename = 'temp_res2',
      verbose=True,
     )
end_multi = time.time()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.8 µs


In [ ]:
end_multi - start_multi

In [ ]:
parameters = ["x", "y"]

In [ ]:
print()
print('evidence: %(logZ).1f +- %(logZerr).1f' % result)
print()
print('parameter values:')
for name, col in zip(parameters, result['samples'].transpose()):
	print('%15s : %.3f +- %.3f' % (name, col.mean(), col.std()))

In [ ]:
import json
with open('%sparams.json' % '.', 'w') as f:
    json.dump(parameters, f, indent=2)

In [ ]:
# solution = PyMultinestStatModel('Xe', 
#                                 n_dims=ndim,
#                                 n_live_points=nlive, 
#                                 evidence_tolerance=tol)

In [ ]:
# solution

In [ ]:
stats2 = dddm.NestedSamplerStatModel('Xe')

stats2.nlive = 200
stats2.verbose = 0

In [ ]:
start_nestle = time.time()
stats2.run_nestle()
end_nestle = time.time()

In [ ]:
end_nestle - start_nestle

In [ ]:
stats2.get_summary()

In [ ]:
%run common_init.py

In [ ]:
stats3 = dddm.NestedSamplerStatModel('Xe')

stats3.nlive = 200
stats3.verbose = 1
stats3.sampler = 'multinest'

In [ ]:
start_nestle = time.time()
stats3.run_multinest()
end_nestle = time.time()

In [ ]:
end_nestle - start_nestle

In [ ]:
from pymultinest import Analyzer

In [ ]:
# create analyzer object
a = Analyzer(2, outputfiles_basename = "results/nested_multinest4/multinest")

# get a dictionary containing information about
#   the logZ and its errors
#   the individual modes and their parameters
#   quantiles of the parameter posteriors
stats = a.get_stats()

# get the best fit (highest likelihood) point
bestfit_params = a.get_best_fit()

# iterate through the "posterior chain"
# for params in a.get_equal_weighted_posterior():
#         print (params)

In [ ]:
a.get_best_fit()

In [ ]:
assert False

## Distribution of the DM ##
First we need to make a DM-rate spectrum

In [ ]:
stats_full = dddm.NestedSamplerStatModel('Xe_migd')

stats_full.nlive = 200
stats_full.verbose = 1
stats_full.sampler = 'multinest'

In [ ]:
stats_full.log
stats_full.set_prior('migdal_lower')

In [ ]:
# for key in stats_full.log.keys():
#     stats_full.log[key] = False


In [ ]:
stats_full.fit_parameters = stats_full.known_parameters

In [ ]:
stats_full.check_spectrum()

In [ ]:
start = time.time()
stats_full.run_multinest()
end = time.time()

In [ ]:
end-start

In [ ]:
stats_full.save_results()

In [ ]:
9170.64813709259/3600

In [ ]:
assert stats_full.log['did_run']

In [ ]:
stats_full.show_walkers()

In [ ]:
stats_full.show_corner()

In [ ]:
corner.corner(
    stats_full.sampler.get_chain(
        flat=True,
        thin = 50,
        discard=int(stats_full.nsteps * 0.2)
    ),
              labels=stats_full.fit_parameters);